In [4]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
import requests
import pickle

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df_train=pd.read_csv("data/train_tweet.csv",index_col=False)
df_test=pd.read_csv("data/test_tweet.csv",index_col=False)
print(len(df_train),len(df_test))

27481 3534


In [5]:
df_train=df_train[['textID', 'text', 'sentiment']]
df=df_train.append(df_test)
len(df)
df=df.dropna()
df=df[['text', 'sentiment']]

In [6]:
df["sentiment"]=df["sentiment"].astype('category')
df["sentiment_target"]=df["sentiment"].cat.codes

In [7]:
df.head()

,text,sentiment,sentiment_target
0,"I`d have responded, if I were going",neutral,1
1,Sooo SAD I will miss you here in San Diego!!!,negative,0
2,my boss is bullying me...,negative,0
3,what interview! leave me alone,negative,0
4,"Sons of ****, why couldn`t they put them on t...",negative,0


In [8]:
x=df["text"].values
y=df["sentiment_target"].values
print(len(x),len(y))

31014 31014


In [9]:
sentences_train,sentences_test,y_train,y_test=train_test_split(x,y,test_size=0.8,random_state =0)


In [10]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(sentences_train[2])
print(X_train[2])

To make a long day short--I did nothing. =D well, I had some friends over they just left (3:55am) We have been drinking since 9.  Smashed
[2, 130, 4, 184, 25, 1048, 1, 129, 317, 158, 76, 1, 70, 73, 221, 122, 58, 22, 211, 110, 3947, 53, 18, 98, 718, 262, 511, 1946]


In [11]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])

[  51    2  675   73   85 3940 1176 3941    2  532  813    2  109 1349
    1  375  128   77   85  406 2571 3942 1177 3943 3944 2572    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [12]:
#model
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1053700   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 128)           64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,119,129
Trainable params: 1,119,129
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)


Train on 6202 samples, validate on 24812 samples
Epoch 1/20
6202/6202 [==============================] - 20s 3ms/step - loss: -1.5380 - acc: 0.3963 - val_loss: -3.0549 - val_acc: 0.4066
Epoch 2/20
6202/6202 [==============================] - 19s 3ms/step - loss: -3.5234 - acc: 0.4813 - val_loss: -3.2325 - val_acc: 0.4811
Epoch 3/20
6202/6202 [==============================] - 19s 3ms/step - loss: -4.0694 - acc: 0.5530 - val_loss: -3.1954 - val_acc: 0.4919
Epoch 4/20
6202/6202 [==============================] - 19s 3ms/step - loss: -4.3138 - acc: 0.5890 - val_loss: -3.1803 - val_acc: 0.4913
Epoch 5/20
6202/6202 [==============================] - 21s 3ms/step - loss: -4.4810 - acc: 0.6111 - val_loss: -3.1235 - val_acc: 0.4910
Epoch 6/20
6202/6202 [==============================] - 21s 3ms/step - loss: -4.5653 - acc: 0.6261 - val_loss: -3.1350 - val_acc: 0.4817
Epoch 7/20
6202/6202 [==============================] - 18s 3ms/step - loss: -4.6208 - acc: 0.6390 - val_loss: -3.0664 - val_acc

NameError: name 'plot_history' is not defined